In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import Geohash
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import logging

def sendDataToDB(iter):
    client = MongoClient()
    db = client.fit5148_db
    ##change collection
    climate = db.climate
    hotspot = db.hotspot
    dclimate=[]
    dhotspots=[]
    # step1: store values in a list
    p2 = []
    p3 = []
    #recognize producer
    for record in iter:
        jsonData = json.loads(record[1])
        if   jsonData["sender_id"] =="producer1":
            dclimate.append(jsonData)
        elif jsonData["sender_id"] =="producer2":
            p2.append(jsonData)
            
        elif jsonData["sender_id"] =="producer3":
            p3.append(jsonData)
            
            #if terra and aqua both have data
        
    if len(p2)>=1 and len(p3)>=1:
        print("all have data")
        for each_p2 in p2:
            for each_p3 in p3:
                try:
                    test1 = Geohash.encode(each_p2["latitude"],each_p2["longitude"],precision=5)
                    test2 = Geohash.encode(each_p3["latitude"],each_p3["longitude"],precision=5)
                    # if terra's location equal to aqua
                    if test1 == test2:
                        fake = {}
                        fake['sender_id']    = [each_p2['sender_id'] ,each_p3['sender_id']]
                        fake['created_time'] = each_p2['created_time']
                        #get avg 
                        fake['longitude']    = (each_p2['longitude'] + each_p3['longitude'])/2
                        fake['confidence']   = (each_p2['confidence']+each_p3['longitude'])/2
                        fake['latitude'] = (each_p2['latitude'] + each_p3['latitude'])/2
                        fake['surface_temperature_celcius'] = (each_p2['surface_temperature_celcius']+each_p3['surface_temperature_celcius'])/2
                        print('fake',fake)
                        dhotspots.append(fake)                       
                    else:
                        if each_p2 not in dhotspots:
                            dhotspots.append(each_p2)
                        if each_p3 not in dhotspots:
                            dhotspots.append(each_p3)
                except:
                    if each_p2 not in dhotspots:
                        dhotspots.append(each_p2)
                    if each_p3 not in dhotspots:
                        dhotspots.append(each_p3)
                    
                    # terra or auqa doesn't have location
    if len(p2)>=1 and len(p3)==0:
        for each_p2 in p2:
            dhotspots.append(each_p2)
    if len(p2)==0 and len(p3)>=1:
        for each_p3 in p3:
            dhotspots.append(each_p3)

            # if climate location equal to hotspots
    tmphot = []
    for clitem in dclimate:
        if len(dhotspots) !=0:
            for hlitem in dhotspots:
                try:
                    test1 = Geohash.encode(clitem["latitude"],clitem["longitude"],precision=5)
                    test2 = Geohash.encode(hlitem["latitude"],hlitem["longitude"],precision=5)
                    if test1 == test2:
                        print('have match')
                        hlitem.pop('longitude')
                        hlitem.pop('latitude')
                        if 'hotspots' in clitem.keys():
                            clitem['hotspots'].append(hlitem)
                        else:
                            x=[]
                            x.append(hlitem)
                            clitem['hotspots'] = x
                    else:
                        if hlitem not in tmphot:
                            tmphot.append(hlitem)
                            hotspot.insert(hlitem)                                                   
                        print('no match')
                except:
                    pass
                    #print('no latitude or longitude in climate or hotspot')
        else:
            print('no hotspots')
        try:
            climate.insert(clitem)
        except:
            pass
        
    client.close()
                
    
    
    #for item in climate_dict.items():
        #taskC2.insert_one(item)        
    
    #tmp = taskAB.find({})
    #for document in tmp[:100]: 
        #pprint(document)


    

#set batch interval to 10 secs
n_secs = 10
topic = "Scenario01"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                    

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))
ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

